In [15]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import DebugStatus
from benchmark_consts import *

In [16]:
options = DebugOptions()
options.is_verus = False

round1 = BenchViewer(UNSTABLE_MARIPOSA, options)

[WARN] [init] 2a3b1202bd trace slow unknown, fallback to fast_fail [WARN] [init] 5b7d18ab3e trace did not fail, fallback to fast_fail [WARN] [init] 3cead21983 trace slow unknown, fallback to fast_fail 


[WARN] [init] e998d08b31 trace slow unknown, fallback to fast_fail 
[WARN] [init] d4e5e77cbc trace slow unknown, fallback to fast_fail 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [init] 0b497bc090 trace did not fail, fallback to fast_fail 
[WARN] [proof] no proofs available 
[WARN] [init] 3ce5c5e4dc trace slow unknown, fallback to fast_fail 


In [17]:
round1.status.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| DebugStatus.FIX_FOUND     |     328 | 60.18 %      |
| DebugStatus.FIX_NOT_FOUND |     213 | 39.08 %      |
| DebugStatus.NO_PROOF      |       4 | 0.73 %       |
| total                     |     545 | 100.00 %     |


In [18]:
for q in round1.status[DebugStatus.NOT_CREATED]:
    print("./src/debugger3.py --not-verus -m auto -i", round1[q].name_hash, "--create-project")

In [19]:
for q in round1.status[DebugStatus.NOT_TESTED]:
    print("./src/make_spaghet.py --not-verus --cluster -i", round1[q].strainer.test_dir)

In [20]:
from utils.cluster_utils import get_sync_commands
from utils.system_utils import write_misc_script

for q in round1.status.tally:
    dbg = round1[q]
    dbg.collect_garbage()

cmds = get_sync_commands("g2001", round1.get_sync_dirs())
write_misc_script("send.sh", cmds)

[WARN] [dbg] skipped GC on unfinished project DebugStatus.NO_PROOF data/projs/bench_unstable/base.z3/fs_vwasm--compiler-linearizer-queries-Compiler.Linearizer-15.smt2 
[WARN] [dbg] skipped GC on unfinished project DebugStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_fvbkv--lib-Base-BitsetLemmas.i.dfy.Impl__BitsetLemmas.__default.all__in__set__implies__all1s__uint64.smt2 
[WARN] [dbg] skipped GC on unfinished project DebugStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_komodo--verified-remove.gen.dfyImpl___module.__default.va__lemma__kom__smc__remove__non__addr__success.smt2 
[WARN] [dbg] skipped GC on unfinished project DebugStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_lvbkv--Betree-Graph.i.dfy.Impl__Graph.__default.LocalEditPreservesAcyclic.smt2 
[INFO] created misc script: send.sh with 3246 commands 


In [23]:
from utils.analysis_utils import fmt_percent

bench_total = len(round1.status.tally)

to_cases = dict()
ff_cases = dict()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

singleton_finished = round1.get_finished()
singleton_fixes = dict()

for q in round1.status.tally:
    dbg = round1[q]
    if dbg.mode == DbgMode.TIMEOUT:
        if q in round1.status[DebugStatus.FIX_FOUND]:
            to_cases[q] = True
            singleton_fixes[q] = dbg.report.stabilized.edit_path.values.tolist()
    elif dbg.mode == DbgMode.FAST_FAIL:
        if q in round1.status[DebugStatus.FIX_FOUND]:
            ff_cases[q] = True
            singleton_fixes[q] = dbg.report.stabilized.edit_path.values.tolist()
    else:
        print("unexpected mode", dbg.mode)
        assert False

print("TOs:", print_break_down(len(to_cases), bench_total))
rd1_to_no_proof = to_cases.keys() & round1.status[DebugStatus.NO_PROOF].items
rd1_to_fixable = sum(to_cases.values())
rd1_to_no_fix = to_cases.keys() & round1.status[DebugStatus.FIX_NOT_FOUND].items
print("\t- 1-no-proof:", print_break_down(len(rd1_to_no_proof), len(to_cases)))
print("\t- 1-fixable:", print_break_down(rd1_to_fixable, len(to_cases)))
print("\t- 1-unfixable:", print_break_down(len(rd1_to_no_fix), len(to_cases)))

print("FFs:", print_break_down(len(ff_cases), bench_total))
rd1_ff_fixable = sum(ff_cases.values())
rd1_ff_no_proof = ff_cases.keys() & round1.status[DebugStatus.NO_PROOF].items
rd1_ff_no_fix = ff_cases.keys() & round1.status[DebugStatus.FIX_NOT_FOUND].items
print("\t- 1-no-proof:", print_break_down(len(rd1_ff_no_proof), len(ff_cases)))
print("\t- 1-fixable:", print_break_down(rd1_ff_fixable, len(ff_cases)))
print("\t- 1-unfixable:", print_break_down(len(rd1_ff_no_fix), len(ff_cases)))

print("")
print("1-fixable:", print_break_down(rd1_to_fixable + rd1_ff_fixable, bench_total))

TOs: 276/545 (50.6%)
	- 1-no-proof: 0/276 (0.0%)
	- 1-fixable: 276/276 (100.0%)
	- 1-unfixable: 0/276 (0.0%)
FFs: 52/545 (9.5%)
	- 1-no-proof: 0/52 (0.0%)
	- 1-fixable: 52/52 (100.0%)
	- 1-unfixable: 0/52 (0.0%)

1-fixable: 328/545 (60.2%)


In [9]:
rd2_to_targets = [round1[q].name_hash for q in rd1_to_no_fix]

options = DebugOptions()
options.is_verus = False
# it is tested under the name DOUBLETON
# overwrite the mode, otherwise AUTO -> TIMEOUT
options.mode = DbgMode.DOUBLETON

round2_to = BenchViewer(rd2_to_targets, options)

In [10]:
round2_to.status.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| DebugStatus.FIX_NOT_FOUND |     133 | 79.17 %      |
| DebugStatus.FIX_FOUND     |      35 | 20.83 %      |
| total                     |     168 | 100.00 %     |


In [11]:
from utils.system_utils import list_smt2_files

rd2_sk_targets = [round1[q].name_hash for q in rd1_ff_no_fix]

# this is a subset of rd2_sk_targets
skolmized = list_smt2_files(MARIPOSA_SKOLEM_DIR)
options = DebugOptions()
options.mode = DbgMode.SKOLEM

round2_sk = BenchViewer(skolmized, options)

[WARN] [proof] no proofs available [WARN] [proof] no proofs available 

[WARN] [proof] no proofs available [WARN] [proof] no proofs available 

[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available [WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 



In [12]:
round2_sk.status.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| DebugStatus.FIX_NOT_FOUND |      19 | 43.18 %      |
| DebugStatus.FIX_FOUND     |      14 | 31.82 %      |
| DebugStatus.NO_PROOF      |      11 | 25.0 %       |
| total                     |      44 | 100.00 %     |


In [13]:
print("TOs:", print_break_down(len(to_cases), bench_total))
rd1_to_no_proof = to_cases.keys() & round1.status[DebugStatus.NO_PROOF].items
rd1_to_fixable = sum(to_cases.values())
rd1_to_no_fix = to_cases.keys() & round1.status[DebugStatus.FIX_NOT_FOUND].items
print("\t- 1-no-proof:", print_break_down(len(rd1_to_no_proof), len(to_cases)))
print("\t- 1-fixable:", print_break_down(rd1_to_fixable, len(to_cases)))
# print("\t- 1-unfixable:", print_break_down(len(rd1_to_no_fix), len(to_cases)))
rd2_to_fixable = round2_to.status[DebugStatus.FIX_FOUND].items
assert rd2_to_fixable.issubset(to_cases.keys())
rd2_to_fixable = len(rd2_to_fixable)
print("\t- 2-fixable:", print_break_down(rd2_to_fixable, len(to_cases)))

print("FFs:", print_break_down(len(ff_cases), bench_total))
rd1_ff_fixable = sum(ff_cases.values())
rd1_ff_no_proof = ff_cases.keys() & round1.status[DebugStatus.NO_PROOF].items
rd1_ff_no_fix = ff_cases.keys() & round1.status[DebugStatus.FIX_NOT_FOUND].items
print("\t- 1-no-proof:", print_break_down(len(rd1_ff_no_proof), len(ff_cases)))
print("\t- 1-fixable:", print_break_down(rd1_ff_fixable, len(ff_cases)))
# print("\t- 1-unfixable:", print_break_down(len(rd1_ff_no_fix), len(ff_cases)))
rd2_ff_fixable = round2_sk.status[DebugStatus.FIX_FOUND].items

for q in rd2_ff_fixable:
    assert round1[round2_sk[q].pre_skolem_name_hash].given_query_path in rd1_ff_no_fix

rd2_ff_fixable = len(rd2_ff_fixable)
print("\t- 2-fixable:", print_break_down(rd2_ff_fixable, len(ff_cases)))

print("")
print("1-fixable:", print_break_down(rd1_to_fixable + rd1_ff_fixable, bench_total))
print("2-fixable:", print_break_down(rd2_to_fixable + rd2_ff_fixable, bench_total))
print("*-fixable:", print_break_down(rd1_to_fixable + rd1_ff_fixable + rd2_to_fixable + rd2_ff_fixable, bench_total))

TOs: 447/545 (82.0%)
	- 1-no-proof: 3/447 (0.7%)
	- 1-fixable: 276/447 (61.7%)
	- 2-fixable: 35/447 (7.8%)
FFs: 98/545 (18.0%)
	- 1-no-proof: 1/98 (1.0%)
	- 1-fixable: 52/98 (53.1%)
	- 2-fixable: 14/98 (14.3%)

1-fixable: 328/545 (60.2%)
2-fixable: 49/545 (9.0%)
*-fixable: 377/545 (69.2%)


In [21]:
print(singleton_fixes)

NameError: name 'singleton_fixes' is not defined